# Etap 4 - Deserializacja

Ostatni etap projektu:

* Wczytanie wytrenowanego modelu.
* Przetesotwanie modelu.

## Spis treści
1. [Import bibliotek i inicjalizacja klienta Dask](#import-bibliotek-i-inicjalizacja-klienta-dask)
2. [Wczytanie modelu](#wczytanie-modelu)
3. [Test modelu](#test-modelu)

## Import bibliotek i inicjalizacja klienta Dask

In [7]:
from xgboost.dask import DaskXGBRegressor
import dask.dataframe as dd

In [1]:
from dask.distributed import LocalCluster, Client

cluster = LocalCluster(
    processes=True,
    n_workers=2,
    threads_per_worker=4,
    memory_limit='12GB'
)

client = Client(cluster)
client

D:\Projekty\BigDataAnalysis\.venv\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59150 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:59150/status,
Dashboard: http://127.0.0.1:59150/status,Workers: 2
Total threads: 8,Total memory: 22.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59153,Workers: 2
Dashboard: http://127.0.0.1:59150/status,Total threads: 8
Started: Just now,Total memory: 22.35 GiB
Comm: tcp://127.0.0.1:59165,Total threads: 4
Dashboard: http://127.0.0.1:59166/status,Memory: 11.18 GiB
Nanny: tcp://127.0.0.1:59156,


## Wczytanie modelu

In [3]:
xgb_dask_reg_loaded = DaskXGBRegressor()
xgb_dask_reg_loaded.load_model('xgb_dask_reg_model_tip_amount.json')

## Test modelu

Model został przetestowany z użyciem danych testowych w poprzednim notebook'u.

In [14]:
def predict_tip(model, payment_type, fare_amount, tolls_amount, extra, trip_time_seconds, trip_distance):
    """
    Przewiduje wysokość napiwka na podstawie wczytanego modelu i podanych parametrów.

    Args:
        model: Wcześniej wczytany model Dask/XGBoost.
        payment_type (int): Typ płatności.
        fare_amount (float): Kwota za przejazd.
        tolls_amount (float): Opłaty drogowe.
        extra (float): Dodatkowe opłaty.
        trip_time_seconds (int): Czas podróży w sekundach.
        trip_distance (float): Dystans podróży w milach.

    Returns:
        float: Przewidywana wysokość napiwka.
    """
        
    input_data = dd.from_dict({
        'payment_type': [payment_type],
        'fare_amount': [fare_amount],
        'tolls_amount': [tolls_amount],
        'extra': [extra],
        'trip_time_seconds': [trip_time_seconds],
        'trip_distance': [trip_distance]
    }, npartitions=1)

    return model.predict(input_data).compute()[0]

In [32]:
round(predict_tip(model=xgb_dask_reg_loaded,
            payment_type=1,
            fare_amount=35.0,
            tolls_amount=1,
            extra=1.5,
            trip_time_seconds=2000,
            trip_distance=12.0
            ), 2)

Windows is not officially supported for dask/xgboost, contribution are welcomed.
Windows is not officially supported for dask/xgboost, contribution are welcomed.


0.06